The dataset used to make the maps here contains some data on SpaceX rocket launches. The 'class' column indicates whether the launch was successful: class = 1 means the launch was successful, and class = 0 means the launch was not successful. There are also 'Lat' and 'Long' columns which indicate the latitude and longitude of launch sites, respectively.

## Setting Up Libraries

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Importing Data

In [2]:
#SpaceX rocket launch data
spacex_df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

In [3]:
#Launch sites data
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


## Map Visualizations

First, let's create a map centered at the NASA Johnson Space Center.

In [4]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

Next, let's add markers for each launch site.

In [5]:
for label, lat, long in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
    circle = folium.Circle([lat, long], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker(
        [lat, long],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label,
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

Now for each launch site, we can add marker clusters containing individual markers for launch records, with green markers indicating successful launches and red markers indicating failed launches. To do so, we first add the 'marker_color' column to `spacex_df`.

In [6]:
def marker_color(x):
    if x == 1:
        marker_color = 'green'
    elif x == 0:
        marker_color = 'red'
    return marker_color
spacex_df['marker_color'] = spacex_df['class'].map(lambda x : marker_color(x))
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [7]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
        )
    marker_cluster.add_child(marker)

site_map

By clicking on each site, one can see the total number of successful launches and failed launches at each site.

Next, let's look at how far away each site is from its nearest city by adding a line segment between each site and its nearby city and markers indicating the distances. To do so, we first need to add mouse position to the map so that we can get the coordinates of the cities by hovering the mouse over the map.

In [8]:
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

By looking at the map, we can see that Cape Canaveral is the closest city to CCAFS LC-40 and CCAFS SLC-40 with latitude 28.38778 and longitude -80.60488, that Titusville is the closest city to KSC LC-39A with latitude 28.61233 and longitude -80.80773, and that Lompoc is the closest city to VAFB SLC-4E with latitude 34.63858 and longitude -120.45767. Let's add this information to `launch_sites_df`.

In [9]:
launch_sites_df['Nearby City'] = ['Cape Canaveral', 'Cape Canaveral', 'Titusville', 'Lompoc']
launch_sites_df['City Lat'] = [28.38778, 28.38778, 28.61233, 34.63858]
launch_sites_df['City Long'] = [-80.60488, -80.60488, -80.80773, -120.45767]
launch_sites_df

,Launch Site,Lat,Long,Nearby City,City Lat,City Long
0,CCAFS LC-40,28.562302,-80.577356,Cape Canaveral,28.38778,-80.60488
1,CCAFS SLC-40,28.563197,-80.576820,Cape Canaveral,28.38778,-80.60488
2,KSC LC-39A,28.573255,-80.646895,Titusville,28.61233,-80.80773
3,VAFB SLC-4E,34.632834,-120.610745,Lompoc,34.63858,-120.45767


Let's also calculate the distances using latitudes and longitudes and then add a 'Distance' column to `launch_sites_df`.

In [10]:
#Define the function to calculate distances
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):
    #Approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
launch_sites_df['Distance'] = list(map(lambda lat1, long1, lat2, long2: calculate_distance(lat1, long1, lat2, long2),
                                  launch_sites_df['Lat'], launch_sites_df['Long'],
                                  launch_sites_df['City Lat'], launch_sites_df['City Long']))
launch_sites_df

,Launch Site,Lat,Long,Nearby City,City Lat,City Long,Distance
0,CCAFS LC-40,28.562302,-80.577356,Cape Canaveral,28.38778,-80.60488,19.597693
1,CCAFS SLC-40,28.563197,-80.576820,Cape Canaveral,28.38778,-80.60488,19.703562
2,KSC LC-39A,28.573255,-80.646895,Titusville,28.61233,-80.80773,16.298083
3,VAFB SLC-4E,34.632834,-120.610745,Lompoc,34.63858,-120.45767,14.023724


Finally, let's add line segments between launch sites and their nearby cities and markers indicating the distances to the map.

In [12]:
for index, record in launch_sites_df.iterrows():
    city_marker = folium.Marker(
        [record['City Lat'], record['City Long']],
        icon=DivIcon(
            icon_size=(30,30),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(record['Distance'])
            )
        )
    site_map.add_child(city_marker)
    city_line = folium.PolyLine(
        locations=[[record['City Lat'], record['City Long']], [record['Lat'], record['Long']]],
        weight=1)
    site_map.add_child(city_line)

site_map

One can zoom in the map around individual launch sites to look at the line segments and distance markers.